In [1]:
#################################### IMPORTS ###################################################
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import datetime
from keras.backend.tensorflow_backend import set_session

Using TensorFlow backend.


In [2]:
##################### INICIALIZACAO DE VARIAVEIS ################################################
index_arquivo = ['preco', 'hr_int']#, 'preco_pon', 'qnt_soma', 'max', 'min', 'IND', 'ISP'] #index do arquivo
steps = [] # 9h04 -> 17h50 a cada 5 segundos 
epocas = 1000 #quantidade de vezes que vai rodar todos os dias
janela = 10 #janela de valores
n_variaveis = len(index_arquivo) #'preco', 'hr_int', 'preco_pon', 'qnt_soma', 'max', 'min', 'IND', 'ISP'
l = n_variaveis-1
tam_janela = n_variaveis*janela
n_entradas = n_variaveis * janela + 2 #ncont, valor, posicao e inputs
n_neuronios = 216 #numero de neuronios da camada escondida
lim_cont = 5
n_saidas = 2*lim_cont+1   #numero de saidas da rede (compra, vende, segura)
custo = 1.06/2 #custo da operao
melhor_reward = 0

versao_arquivo = 2

carregar_pesos = True
carregar_epoca_epsilon = False
epoca_init = 0
if carregar_epoca_epsilon:
    file = open("./epoca_epsilon.txt", "r")
    valores = file.read().split(',')
    epoca_init = int(valores[0])
    epsilon = float(valores[1])
    file.close()
else:
    epsilon = 1. #valor de epsilon
epsilon_min = 0.01 #valor minimo de epsilon
epsilon_decay = (epsilon - epsilon_min) / (epocas - epoca_init) #o valor que vai retirado do epsilon por epoca

rewards = [0] #variavel para guardar rewards
plotx = [0] #variavel para guardar valores a serem plotados do eixo x

In [3]:
####################### LEITURA DOS DADOS #######################################################
dias = 220 # dias_treino - 220 dias de 284 ~ 77,5% dos dados para treino

caminho_arquivo = ('./consolidado.csv')
arquivo = pd.read_csv(caminho_arquivo) #le arquivo
inputs = arquivo[index_arquivo]
print(inputs['preco'][0:2])
if versao_arquivo == 1: #se quiser usar apenas os dias com IND e ISP
    inputs = inputs[inputs['IND'] != 0]
    arquivo = arquivo[arquivo['IND'] != 0]

dt = arquivo['dt'].values #cria coluna apenas dos dias

steps = []
ultimo_dia = 0
dias_para_rodar = [] #variavel para colocar os dias a serem rodados
j = 0
hr = []
h = 0

for i in range( 0, len(dt) ):    
    if (dt[i] != ultimo_dia):
        steps.append(i) #numero de linhas entre dias
        ultimo_dia = dt[i]
        dias_para_rodar.append(j) #numero do dia
        j += 1
        h = 0
    hr.append(h)
    h += 1
step_max = np.amax(hr)
batch = step_max + 1

#normalizacao dos dados
hr = hr/step_max
inputs['hr_int'] = hr
pmean = np.mean( inputs.loc[:dias*batch, inputs.columns[0]] ) #define valor minimo do preco  #step = 106
pstd = np.std( inputs.loc[:dias*batch, inputs.columns[0]] ) #define valor maximo do preco
for i in range( inputs.shape[1] ): #roda normalizo para todas as colunas
    imean = np.mean( inputs.loc[:dias*batch, inputs.columns[i]] ) #pega valor maximo
    istd = np.std( inputs.loc[:dias*batch, inputs.columns[i]] ) #pega valor minimo
    
    inputs.loc[:, inputs.columns[i]] = ( inputs.loc[:, inputs.columns[i]] - imean ) / istd #normaliza prs
    
mini_batch_size = int(dias*batch*0.1)

D:\anaconda\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0    3907.5
1    3900.0
Name: preco, dtype: float64


D:\anaconda\lib\site-packages\pandas\core\indexing.py:635: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [4]:
dias_teste = int(len(dias_para_rodar)*0.2) #20% dos dados para validação

In [5]:
########################################   BIBLIOTECAS ####################################
import random
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras import backend as K            #importa backend para clear_session()


##################### MODELO DQN ####################################################
class DQNAgent:
    ########################### INICIALIZA ###########################################
    def __init__(self, state_size, action_size, epsilon, janela, n_neuronios, n_variaveis):
        self.state_size = state_size
        self.n_neuronios = n_neuronios
        self.action_size = action_size
        self.janela = janela
        self.n_variaveis = n_variaveis
        self.limpa_memoria_dia()
        self.limpa_memoria_epoca()
        self.gamma = 0.97       # discount rate
        self.epsilon = epsilon  # exploration rate
        self.learning_rate = 1e-4
        self.model = self.cria_modelo()
        self.model.summary()
        self.state = []
        self.next_state = []
        self.janela_precos = []
        self.prox_janela_precos = []

################################# REDE NEURAL ###########################################
    def cria_modelo(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        """
        model.add(Dense(self.n_neuronios, input_dim=self.state_size, activation='relu')) #camada de entrada (escondida)
        model.add(Dense(self.n_neuronios, activation='relu')) #camada escondida
        model.add(Dense(self.n_neuronios, activation='relu')) #camada escondida
        model.add(Dense(self.action_size, activation='softmax')) #camada de saida
        model.compile(loss='categorical_crossentropy', optimizer=tf.train.AdamOptimizer(learning_rate=self.learning_rate)) #compilador
        """
        
        model.add(Dense(320, input_dim=self.state_size, activation='relu')) #camada de entrada (escondida)
        model.add(Dense(256, activation='relu')) #camada escondida
        model.add(Dense(256, activation='relu')) #camada escondida
        model.add(Dense(256, activation='relu')) #camada escondida
        model.add(Dense(self.action_size, activation='linear')) #camada de saida
        #model.compile(loss='mse', optimizer=tf.train.AdamOptimizer(learning_rate=self.learning_rate)) #compilador
        sgd = optimizers.SGD(lr=self.learning_rate, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='mean_squared_error', optimizer=sgd)
        return model

    def limpa_memoria_dia(self):
        self.janela_precos = np.zeros(n_variaveis*janela)
        self.prox_janela_precos = np.zeros(n_variaveis*janela)
        self.state = []
        self.next_state = []
    
    def limpa_memoria_epoca(self):
        self.memory = []
        
    def toma_acao(self):     
        act_values = self.model.predict(modelo.state) #calcula qual a melhor acao
        return np.argmax(act_values[0])  # returns action
    
    def treina_modelo(self, batch_size=mini_batch_size):
        minibatch = random.sample(self.memory, batch_size)
        
        x = np.zeros((batch_size+1, self.state_size))
        y = np.zeros((batch_size+1, self.action_size))
        i = 0
        for acao, reward, estado, prox_estado, done in minibatch:        
            
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(prox_estado)[0])) #pega valor que quer chegar

            target_f = self.model.predict(estado) #pega valor que chegou
            target_f[0][acao+lim_cont] = target #define o valor que deseja chegar
            x[i,:] = estado
            y[i,:] = target_f
            i = i + 1
            
        self.model.fit(x, y, epochs=1, verbose=0) #treina modelo
        
    def tira_ultima_janela(self):
        if self.janela_precos.shape[0] > self.janela * self.n_variaveis:
            self.janela_precos = self.janela_precos[self.n_variaveis:] #tira os ultimos precos
    
    def remember(self, acao, reward, done):
        self.memory.append((acao, reward, self.state, self.next_state, done))
    
    def carrega_pesos(self, name):
        self.model.load_weights(name) #carrega pesos

    def salva_pesos(self, name):
        self.model.save_weights(name) #salva pesos
                
########################  DECLARA MODELO ################################
modelo = DQNAgent(n_entradas, n_saidas, epsilon, janela, n_neuronios, n_variaveis)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 320)               7360      
_________________________________________________________________
dense_2 (Dense)              (None, 256)               82176     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 11)                2827      
Total params: 223,947
Trainable params: 223,947
Non-trainable params: 0
_________________________________________________________________


In [8]:
########################### FUNCOES ###############################################################

def atuacao( preco, ncont, acao, custo, valor ):  #preo atual, n de contratos posicionados,
                                                #acaoo atual, custo, valor da posio
    preco_cheio = 0.
    valor_cheio = 0.
    ncont_anterior = ncont #salva posio anterior
    ncont = acao #posicao atual = acao
    reward = 0.
    posicao = 0.
    dp = 0.

    
    #variaveis para tratamento do sinal de ncont, ncont_anterior e da acao
    acao = ncont - ncont_anterior  #ajuda a manter as condições anteriores
    var1 = ncont*ncont_anterior
    var2 = ncont*acao

    if acao != 0:
        #realizacao de lucros e custos
        if ((var2>0) and (var1>=0)):
            #aumento da posicao
            reward = -custo*abs(acao)    #reward = -custo*acao se houve operacao (aumento da posicao)
            #aumento da posicao = preco medio:
            valor = (ncont_anterior*valor + acao*preco)/ncont  
            
        else:
            valor_cheio = valor * pstd + pmean  #valores nao normalizados
            preco_cheio = preco * pstd + pmean
            if (var1)>= 0:
                #diminuicao da posicao
                reward = acao*(valor_cheio - preco_cheio)*10 - custo*abs(acao)  #reward se houve diminuicao da posicao
                #valor medio nao muda na diminuicao, somente se ncont==0:
                if ncont==0: valor=-10
                    
            else:
                #troca de posicao
                reward = (-ncont_anterior)*(valor_cheio - preco_cheio)*10 - custo*abs(acao)  #reward se houve troca da posicao
                #preco medio da posicao = preco da troca de posicao
                valor = preco
    
    #definindo o lucro potencial da carteira(posicao)
    if valor!=-10:
        valor_cheio = valor * pstd + pmean  #valor posicionado atual      
        dp = (preco * pstd + pmean) - valor_cheio #variacao do preco atual e do preco de compra/venda
        posicao = ncont * dp * 10 - custo*abs(ncont)           #posicao = lucro (POTENCIAL)
        
    return ncont, valor, posicao, reward

def obter_acao(ncont):
    decisao = modelo.toma_acao() #calcula a saida da rede neural
    return (decisao - lim_cont)

def rodar_1dia(precos, custo, dia):
    global melhor_reward
    ncont = 0 #cria variavel de quantidade de contratos
    ncont_anterior = 0 #cria variavel para quantidade de contratos anterior
    valor = -10 #cria variavel para preo medio
    reward = 0. #cria variavel para recompensa
    lucro = 0.
    posicao = 0 #cria variavel de posio 
    erro = []
    modelo.limpa_memoria_dia() #limpa o vetor de memoria
    done = False
    posicao_max = 10e4
    shp = modelo.janela_precos.shape
    
    for step in range( steps[dia - 1], steps[dia] ):  #roda os dados
        
        ultimos_precos = precos[ step : step + 1 ] #pega os valores de agora
        modelo.janela_precos = np.delete(np.insert(modelo.janela_precos,shp,np.concatenate(ultimos_precos.values).tolist()),[0,l])
        modelo.state = np.reshape(np.concatenate((modelo.janela_precos, [ncont/lim_cont, valor])), (1,n_entradas)) #adiciona na variavel de estado      
        
        acao = obter_acao(ncont)        
        ncont, valor, posicao, reward = atuacao(precos['preco'][step], ncont, acao, custo, valor)
        
        #pos acao
        lucro += reward    #LUCRO ACUMULADO
        
        v=valor if valor==-10 else valor*pstd+pmean
        p=precos['preco'][step]*pstd+pmean
        print("acao={0}; ncont={1}; valor={2}; preco={3}; posicao={4}; lucro={5}; reward={6}".format(acao, ncont, v, p, posicao, lucro, reward))    
            
    #fim do dia
    lucro += posicao    
    
    if lucro > melhor_reward:
        melhor_reward = lucro
    return lucro #retorna o valor do reward

dias_pos = 0
dias_neg = 0

def rodar_dias(precos, custo):   
    global dias_pos
    global dias_neg
    
    sum_rewards = 0 #cria variavel de somatoria de recompensas
    
    for dia in range( len(dias_para_rodar)-dias_teste+1, len(dias_para_rodar) ): #loop de dias
        reward = rodar_1dia(precos, custo, dia)
        print("Dia {0} resultado: R$ {1:0.2f}\n".format(dia, reward))
        sum_rewards += reward #roda 1 dia e adiciona o total na variavel de somatoria
        
        if reward>0: 
            dias_pos += 1
        elif reward<0:
            dias_neg += 1
    return sum_rewards


In [9]:
import time
##################################  MAIN  #########################################
if __name__ == "__main__":
    global dias_pos
    global dias_neg
    try:
        modelo.carrega_pesos('./pesos_5.h5')
        modelo.limpa_memoria_epoca()
        sum_rewards = rodar_dias(inputs, custo) #adiciona o resultado da epoca na somatoria
        print("resultado do teste = {0:0.2f}".format(sum_rewards))
        print(dias_pos)
        print(dias_neg)
    finally:
        print("Melhor resultado diario: {0:0.2f}".format(melhor_reward))

acao=-5; ncont=-5; valor=3868.0; preco=3868.0; posicao=-2.6500000000000004; lucro=-2.6500000000000004; reward=-2.6500000000000004
acao=2; ncont=2; valor=3868.5; preco=3868.5; posicao=-1.06; lucro=-31.36; reward=-28.71
acao=-5; ncont=-5; valor=3868.0; preco=3868.0; posicao=-2.6500000000000004; lucro=-45.07; reward=-13.71
acao=-5; ncont=-5; valor=3868.0; preco=3870.5; posicao=-127.65; lucro=-45.07; reward=0.0
acao=2; ncont=2; valor=3874.5; preco=3874.5; posicao=-1.06; lucro=-373.78; reward=-328.71
acao=2; ncont=2; valor=3874.5; preco=3874.5; posicao=-1.06; lucro=-373.78; reward=0.0
acao=2; ncont=2; valor=3874.5; preco=3874.0; posicao=-11.06; lucro=-373.78; reward=0.0
acao=2; ncont=2; valor=3874.5; preco=3873.0; posicao=-31.06; lucro=-373.78; reward=0.0
acao=2; ncont=2; valor=3874.5; preco=3873.0; posicao=-31.06; lucro=-373.78; reward=0.0
acao=2; ncont=2; valor=3874.5; preco=3873.5; posicao=-21.06; lucro=-373.78; reward=0.0
acao=2; ncont=2; valor=3874.5; preco=3871.0; posicao=-71.06; lucr

acao=2; ncont=2; valor=3815.0; preco=3823.5; posicao=168.94; lucro=391.22; reward=0.0
acao=2; ncont=2; valor=3815.0; preco=3827.0; posicao=238.94; lucro=391.22; reward=0.0
acao=2; ncont=2; valor=3815.0; preco=3823.5; posicao=168.94; lucro=391.22; reward=0.0
acao=2; ncont=2; valor=3815.0; preco=3823.0; posicao=158.94; lucro=391.22; reward=0.0
acao=2; ncont=2; valor=3815.0; preco=3824.0; posicao=178.94; lucro=391.22; reward=0.0
acao=2; ncont=2; valor=3815.0; preco=3823.0; posicao=158.94; lucro=391.22; reward=0.0
acao=2; ncont=2; valor=3815.0; preco=3822.0; posicao=138.94; lucro=391.22; reward=0.0
acao=2; ncont=2; valor=3815.0; preco=3821.5; posicao=128.94; lucro=391.22; reward=0.0
acao=2; ncont=2; valor=3815.0; preco=3821.5; posicao=128.94; lucro=391.22; reward=0.0
acao=2; ncont=2; valor=3815.0; preco=3819.5; posicao=88.94; lucro=391.22; reward=0.0
acao=2; ncont=2; valor=3815.0; preco=3824.0; posicao=178.94; lucro=391.22; reward=0.0
acao=2; ncont=2; valor=3815.0; preco=3822.5; posicao=14

acao=-5; ncont=-5; valor=3854.0; preco=3851.5; posicao=122.35; lucro=80.09000000000005; reward=0.0
acao=-5; ncont=-5; valor=3854.0; preco=3852.5; posicao=72.35; lucro=80.09000000000005; reward=0.0
acao=-5; ncont=-5; valor=3854.0; preco=3852.5; posicao=72.35; lucro=80.09000000000005; reward=0.0
acao=-5; ncont=-5; valor=3854.0; preco=3849.0; posicao=247.35; lucro=80.09000000000005; reward=0.0
acao=-5; ncont=-5; valor=3854.0; preco=3848.0; posicao=297.35; lucro=80.09000000000005; reward=0.0
acao=-5; ncont=-5; valor=3854.0; preco=3847.0; posicao=347.35; lucro=80.09000000000005; reward=0.0
acao=-5; ncont=-5; valor=3854.0; preco=3847.0; posicao=347.35; lucro=80.09000000000005; reward=0.0
acao=-5; ncont=-5; valor=3854.0; preco=3847.5; posicao=322.35; lucro=80.09000000000005; reward=0.0
acao=-5; ncont=-5; valor=3854.0; preco=3848.0; posicao=297.35; lucro=80.09000000000005; reward=0.0
acao=-5; ncont=-5; valor=3854.0; preco=3844.0; posicao=497.35; lucro=80.09000000000005; reward=0.0
acao=-5; nco

acao=2; ncont=2; valor=3831.0; preco=3825.0; posicao=-121.06; lucro=236.22000000000003; reward=0.0
acao=2; ncont=2; valor=3831.0; preco=3822.5; posicao=-171.06; lucro=236.22000000000003; reward=0.0
acao=2; ncont=2; valor=3831.0; preco=3823.5; posicao=-151.06; lucro=236.22000000000003; reward=0.0
acao=2; ncont=2; valor=3831.0; preco=3825.5; posicao=-111.06; lucro=236.22000000000003; reward=0.0
acao=2; ncont=2; valor=3831.0; preco=3825.5; posicao=-111.06; lucro=236.22000000000003; reward=0.0
acao=2; ncont=2; valor=3831.0; preco=3824.5; posicao=-131.06; lucro=236.22000000000003; reward=0.0
acao=2; ncont=2; valor=3831.0; preco=3826.0; posicao=-101.06; lucro=236.22000000000003; reward=0.0
acao=2; ncont=2; valor=3831.0; preco=3825.0; posicao=-121.06; lucro=236.22000000000003; reward=0.0
acao=2; ncont=2; valor=3831.0; preco=3825.0; posicao=-121.06; lucro=236.22000000000003; reward=0.0
acao=2; ncont=2; valor=3831.0; preco=3825.0; posicao=-121.06; lucro=236.22000000000003; reward=0.0
acao=2; nc

acao=-5; ncont=-5; valor=3852.5; preco=3837.0; posicao=772.35; lucro=396.90999999999997; reward=0.0
acao=-5; ncont=-5; valor=3852.5; preco=3837.5; posicao=747.35; lucro=396.90999999999997; reward=0.0
acao=-5; ncont=-5; valor=3852.5; preco=3840.0; posicao=622.35; lucro=396.90999999999997; reward=0.0
acao=-5; ncont=-5; valor=3852.5; preco=3842.0; posicao=522.35; lucro=396.90999999999997; reward=0.0
acao=-5; ncont=-5; valor=3852.5; preco=3842.5; posicao=497.35; lucro=396.90999999999997; reward=0.0
acao=-5; ncont=-5; valor=3852.5; preco=3837.0; posicao=772.35; lucro=396.90999999999997; reward=0.0
acao=-5; ncont=-5; valor=3852.5; preco=3838.5; posicao=697.35; lucro=396.90999999999997; reward=0.0
acao=-5; ncont=-5; valor=3852.5; preco=3833.0; posicao=972.35; lucro=396.90999999999997; reward=0.0
acao=-5; ncont=-5; valor=3852.5; preco=3826.5; posicao=1297.35; lucro=396.90999999999997; reward=0.0
acao=-5; ncont=-5; valor=3852.5; preco=3830.0; posicao=1122.35; lucro=396.90999999999997; reward=0.

...

acao=-2; ncont=-2; valor=4027.0; preco=4023.0; posicao=78.94; lucro=737.5999999999999; reward=0.0
acao=-2; ncont=-2; valor=4027.0; preco=4019.5; posicao=148.94; lucro=737.5999999999999; reward=0.0
acao=-5; ncont=-5; valor=4024.3; preco=4022.5; posicao=87.35000000000909; lucro=736.0099999999999; reward=-1.59
acao=-5; ncont=-5; valor=4024.3; preco=4023.0; posicao=62.350000000009096; lucro=736.0099999999999; reward=0.0
acao=-5; ncont=-5; valor=4024.3; preco=4025.5; posicao=-62.649999999990904; lucro=736.0099999999999; reward=0.0
acao=-5; ncont=-5; valor=4024.3; preco=4026.5; posicao=-112.64999999999091; lucro=736.0099999999999; reward=0.0
acao=-5; ncont=-5; valor=4024.3; preco=4024.5; posicao=-12.649999999990905; lucro=736.0099999999999; reward=0.0
acao=-5; ncont=-5; valor=4024.3; preco=4028.5; posicao=-212.6499999999909; lucro=736.0099999999999; reward=0.0
acao=-5; ncont=-5; valor=4024.3; preco=4029.0; posicao=-237.6499999999909; lucro=736.0099999999999; reward=0.0
acao=-5; ncont=-5; val

acao=2; ncont=2; valor=4111.0; preco=4118.0; posicao=138.94; lucro=-156.36; reward=0.0
acao=2; ncont=2; valor=4111.0; preco=4116.0; posicao=98.94; lucro=-156.36; reward=0.0
acao=2; ncont=2; valor=4111.0; preco=4112.5; posicao=28.94; lucro=-156.36; reward=0.0
acao=2; ncont=2; valor=4111.0; preco=4112.0; posicao=18.94; lucro=-156.36; reward=0.0
acao=-2; ncont=-2; valor=4110.5; preco=4110.5; posicao=-1.06; lucro=-168.48000000000002; reward=-12.120000000000001
acao=-2; ncont=-2; valor=4110.5; preco=4115.5; posicao=-101.06; lucro=-168.48000000000002; reward=0.0
acao=-2; ncont=-2; valor=4110.5; preco=4116.5; posicao=-121.06; lucro=-168.48000000000002; reward=0.0
acao=-2; ncont=-2; valor=4110.5; preco=4125.5; posicao=-301.06; lucro=-168.48000000000002; reward=0.0
acao=-2; ncont=-2; valor=4110.5; preco=4129.0; posicao=-371.06; lucro=-168.48000000000002; reward=0.0
acao=-2; ncont=-2; valor=4110.5; preco=4132.0; posicao=-431.06; lucro=-168.48000000000002; reward=0.0
acao=-2; ncont=-2; valor=4110

acao=-2; ncont=-2; valor=4157.0; preco=4158.5; posicao=-31.06; lucro=388.35999999999996; reward=0.0
acao=-2; ncont=-2; valor=4157.0; preco=4157.5; posicao=-11.06; lucro=388.35999999999996; reward=0.0
acao=-2; ncont=-2; valor=4157.0; preco=4158.0; posicao=-21.06; lucro=388.35999999999996; reward=0.0
acao=-2; ncont=-2; valor=4157.0; preco=4155.0; posicao=38.94; lucro=388.35999999999996; reward=0.0
acao=-2; ncont=-2; valor=4157.0; preco=4152.5; posicao=88.94; lucro=388.35999999999996; reward=0.0
acao=-2; ncont=-2; valor=4157.0; preco=4152.0; posicao=98.94; lucro=388.35999999999996; reward=0.0
acao=-2; ncont=-2; valor=4157.0; preco=4150.5; posicao=128.94; lucro=388.35999999999996; reward=0.0
acao=-2; ncont=-2; valor=4157.0; preco=4151.5; posicao=108.94; lucro=388.35999999999996; reward=0.0
acao=-2; ncont=-2; valor=4157.0; preco=4154.5; posicao=48.94; lucro=388.35999999999996; reward=0.0
acao=-2; ncont=-2; valor=4157.0; preco=4151.0; posicao=118.94; lucro=388.35999999999996; reward=0.0
acao

acao=-5; ncont=-5; valor=4143.3; preco=4142.5; posicao=37.350000000009096; lucro=-748.2300000000001; reward=-1.59
acao=-5; ncont=-5; valor=4143.3; preco=4143.5; posicao=-12.649999999990905; lucro=-748.2300000000001; reward=0.0
acao=-5; ncont=-5; valor=4143.3; preco=4146.5; posicao=-162.6499999999909; lucro=-748.2300000000001; reward=0.0
acao=-5; ncont=-5; valor=4143.3; preco=4148.0; posicao=-237.6499999999909; lucro=-748.2300000000001; reward=0.0
acao=-5; ncont=-5; valor=4143.3; preco=4148.5; posicao=-262.6499999999909; lucro=-748.2300000000001; reward=0.0
acao=-5; ncont=-5; valor=4143.3; preco=4147.0; posicao=-187.6499999999909; lucro=-748.2300000000001; reward=0.0
acao=-5; ncont=-5; valor=4143.3; preco=4147.5; posicao=-212.6499999999909; lucro=-748.2300000000001; reward=0.0
acao=-5; ncont=-5; valor=4143.3; preco=4146.0; posicao=-137.6499999999909; lucro=-748.2300000000001; reward=0.0
acao=-5; ncont=-5; valor=4143.3; preco=4148.5; posicao=-262.6499999999909; lucro=-748.2300000000001; 

acao=-2; ncont=-2; valor=4157.0; preco=4130.0; posicao=538.94; lucro=-78.48000000000002; reward=0.0
acao=-2; ncont=-2; valor=4157.0; preco=4123.5; posicao=668.94; lucro=-78.48000000000002; reward=0.0
acao=-2; ncont=-2; valor=4157.0; preco=4121.0; posicao=718.94; lucro=-78.48000000000002; reward=0.0
acao=-2; ncont=-2; valor=4157.0; preco=4122.5; posicao=688.94; lucro=-78.48000000000002; reward=0.0
acao=-2; ncont=-2; valor=4157.0; preco=4124.5; posicao=648.94; lucro=-78.48000000000002; reward=0.0
acao=-2; ncont=-2; valor=4157.0; preco=4122.0; posicao=698.94; lucro=-78.48000000000002; reward=0.0
acao=-2; ncont=-2; valor=4157.0; preco=4120.5; posicao=728.94; lucro=-78.48000000000002; reward=0.0
acao=-2; ncont=-2; valor=4157.0; preco=4122.5; posicao=688.94; lucro=-78.48000000000002; reward=0.0
acao=-2; ncont=-2; valor=4157.0; preco=4124.0; posicao=658.94; lucro=-78.48000000000002; reward=0.0
acao=-2; ncont=-2; valor=4157.0; preco=4125.0; posicao=638.94; lucro=-78.48000000000002; reward=0.0
